<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 2



### 2.1 Linear Regression in 1D Forward Prediction



#### Linear Regression



$\hat y =b+wx$



In [ ]:
import torch



In [ ]:
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(-1.0, requires_grad=True)



In [ ]:
def forward(x):
  y = w * x + b
  return y



In [ ]:
x = torch.tensor([1.0])



In [ ]:
yhat = forward(x)
yhat



$x =
  \begin{pmatrix}
    1 \\
    2
  \end{pmatrix}$



In [ ]:
x = torch.tensor(
    [
        [1.0],
        [2.0]
    ],
)



In [ ]:
forward(x)



#### `Linear` class



In [ ]:
import torch.nn

torch.manual_seed(1)



$\hat y_{1 \times 1} =b_{1 \times 1}+w_{1 \times 1}x_{1 \times 1}$



In [ ]:
model = torch.nn.Linear(in_features=1, out_features=1)



$b_{1 \times 1}$ and $w_{1 \times 1}$



In [ ]:
for p in model.parameters():
  print(p)



In [ ]:
x = torch.tensor([0.0])
yhat = model(x)
yhat



$x =
  \begin{pmatrix}
    1 \\
    2
  \end{pmatrix}$



In [ ]:
x = torch.tensor(
    [
        [1.0],
        [2.0]
    ]
)



In [ ]:
yhat = model(x)
yhat



#### Custom Modules
* may include multiple models


In [ ]:
import torch.nn



In [ ]:
class LR(torch.nn.Module):
  def __init__(self, in_size, output_size):
    super(LR, self).__init__()
    self.linear = torch.nn.Linear(in_size, output_size)

  def forward(self, x):
    return self.linear(x)



In [ ]:
model = LR(1, 1)

state = model.state_dict()

weight = state['linear.weight']
weight.data[0] = torch.tensor([0.5153])

bias = state['linear.bias']
bias.data[0] = torch.tensor([-0.4414])



$b_{1 \times 1}$ and $w_{1 \times 1}$



In [ ]:
for p in model.parameters():
  print(p)



In [ ]:
x = torch.tensor([1.0])
model(x)



In [ ]:
x = torch.tensor(
    [
        [1.0],
        [2.0]
    ]
)
model(x)



In [ ]:
state = model.state_dict()
state



### 2.2 Linear Regression in 1D Training



#### Linear Regression



Loss function
$$
l(w, b)=\frac{1}{N}
  \sum_{n=1}^N
    \left(
      y_n - \left(
          wx_n + b
        \right)
    \right)^2
$$



#### Gradient Descent
$$
w^{k+1}=w^{k}-η \frac{d}{dw}l\left(w^k\right)
$$

symbol | description
:------:|-----
$w$ | weight
$l(w)$ | loss as a function of weigth
$$\frac{d}{dw}l(w)$$ | slope of loss as a function of weigth
$k$ | training step
$\eta$ | **learning rate**

